In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('../input/playground-series-s4e5/train.csv')
test = pd.read_csv('../input/playground-series-s4e5/test.csv')

trainDropped = train.drop('id', axis=1) 
testDropped = test.drop('id', axis=1) 

target = "FloodProbability"
randomState = 41
X = trainDropped.drop(target, axis=1)
y = trainDropped[target]

scaler = StandardScaler()
scaler.fit(X)

X = pd.DataFrame(scaler.transform(X), columns=X.columns)

In [3]:
from catboost import Pool
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=randomState)

kf = KFold(n_splits=5, shuffle=True, random_state=randomState)
catboost_params ={'depth': 3,                   
                  'random_state':randomState,
                  'task_type': 'CPU', 
                  'eval_metric': 'RMSE', 
                  'min_data_in_leaf': 4, 
                  'loss_function': 'RMSE',
                  'grow_policy': 'Lossguide', 
                  'bootstrap_type': 'Bernoulli',
                  'subsample': 0.83862137638162, 
                  'l2_leaf_reg': 8.365422739510098, 
                  'random_strength': 3.296124856352495, 
                  'learning_rate': 0.0983, 
}
    
# # データセットの作成。Poolで説明変数、目的変数
# train_pool = Pool(X_train, y_train)
# validate_pool = Pool(X_test, y_test)   

model = CatBoostRegressor(**catboost_params)
va_pred = model.fit(
            X_train,
            y_train,
            eval_set=(X_test, y_test),    # 検証用データ
            early_stopping_rounds=10,  # 10回以上精度が改善しなければ中止
        )

0:	learn: 0.0506656	test: 0.0507337	best: 0.0507337 (0)	total: 278ms	remaining: 4m 37s
1:	learn: 0.0503264	test: 0.0503958	best: 0.0503958 (1)	total: 434ms	remaining: 3m 36s
2:	learn: 0.0499839	test: 0.0500551	best: 0.0500551 (2)	total: 619ms	remaining: 3m 25s
3:	learn: 0.0496399	test: 0.0497096	best: 0.0497096 (3)	total: 772ms	remaining: 3m 12s
4:	learn: 0.0493153	test: 0.0493891	best: 0.0493891 (4)	total: 932ms	remaining: 3m 5s
5:	learn: 0.0489861	test: 0.0490576	best: 0.0490576 (5)	total: 1.1s	remaining: 3m 3s
6:	learn: 0.0486648	test: 0.0487430	best: 0.0487430 (6)	total: 1.26s	remaining: 2m 58s
7:	learn: 0.0483542	test: 0.0484285	best: 0.0484285 (7)	total: 1.41s	remaining: 2m 54s
8:	learn: 0.0480481	test: 0.0481262	best: 0.0481262 (8)	total: 1.57s	remaining: 2m 52s
9:	learn: 0.0477476	test: 0.0478271	best: 0.0478271 (9)	total: 1.72s	remaining: 2m 49s
10:	learn: 0.0474549	test: 0.0475352	best: 0.0475352 (10)	total: 1.87s	remaining: 2m 47s
11:	learn: 0.0471654	test: 0.0472439	best: 0

In [5]:
pred = model.predict(X_test)
score = r2_score(y_test, pred)
print(f'r2_score: {score: .4f}')

r2_score:  0.8416


In [6]:
submit = test[['id']]
testScaled = pd.DataFrame(scaler.transform(testDropped), columns=testDropped.columns)
submit['FloodProbability'] = model.predict(testScaled)
submit
print(submit)

             id  FloodProbability
0       1117957          0.569133
1       1117958          0.461264
2       1117959          0.451531
3       1117960          0.467784
4       1117961          0.468260
...         ...               ...
745300  1863257          0.474199
745301  1863258          0.448473
745302  1863259          0.624518
745303  1863260          0.551339
745304  1863261          0.516284

[745305 rows x 2 columns]


/tmp/ipykernel_18/1089481222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['FloodProbability'] = model.predict(testScaled)


In [7]:
submit.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
